## Create your own Observational Large Ensemble

This code creates an Observational Large Ensemble based on the statistics of the observations and the forced trend from the NCAR CESM1 LENS.  

A limited set of results is discussed in McKinnon and Deser, currently submitted to Journal of Climate. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from netCDF4 import Dataset, MFDataset
from mpl_toolkits.basemap import Basemap, cm, maskoceans, shiftgrid
import glob
import os

### helpful functions

In [ ]:
def autocorr(x):
    x = x - np.mean(x)
    result = np.correlate(x, x, mode = 'full')
    result /= result[np.shape(x)[0] - 1]
    return result[result.size/2:]

In [ ]:
def calcFDR(lon, lat, obsRatio, nullRatio, FDR, testType, domask):
    # Calculate gridbox-wise significance based on controlling the False Discovery Rate 
    # see, e.g., Wilks, 2016, BAMS
    
    # get ocean mask
    if domask:
        var = obsRatio
        if np.max(lon) > 180:
            lon2 = lon
            lon2[lon2 > 180] -= 360
            var2 = var
        else:
            var2 = var
            lon2 = lon
        lons, lats = np.meshgrid(lon2, lat)
        var3 = maskoceans(lons, lats, var2, resolution = 'l')

        obsRatio = np.ma.masked_where(np.ma.getmask(var3), obsRatio)
        maskMat = np.repeat(obsRatio.mask[np.newaxis, :], np.shape(nullRatio)[0], axis = 0)
        nullRatio = np.ma.masked_where(maskMat, nullRatio)

    nmembers = np.shape(nullRatio)[0]
    
    pvalA = np.sum(nullRatio > obsRatio[np.newaxis, :], axis = 0)/float(nmembers)
    pvalB = np.sum(nullRatio < obsRatio[np.newaxis, :], axis = 0)/float(nmembers)
    
    if testType == 'upper': # we expect our value to be higher than the null
        pval = pvalA
    elif testType == 'lower': # we expect our value to be lower than the null
        pval = pvalB
    elif testType == 'both': # we don't know and want to do a two-sided test
        pval = 2.*np.min(np.vstack((pvalA, pvalB)), axis = 0) 
        # need to remask for some reason
        if domask:
            pval = np.ma.masked_where(np.ma.getmask(var3), pval)

    # pval = np.sum(nullRatio > obsRatio[np.newaxis, :], axis = 0)/float(nmembers)

    pvalSorted = np.sort(pval)

    if domask:
        N = float(np.sum(~np.ma.getmask(pvalSorted)))
    else:
        N = float(np.shape(pvalSorted)[0])
    
    cutoff = np.arange(1, np.shape(pval)[0] + 1)/N*FDR
    
    cutoffIdx = (np.where(pvalSorted > cutoff))[0][0]
    print pvalSorted[cutoffIdx]
    
    notSigVals = pval > pvalSorted[cutoffIdx]
        
    return notSigVals, pval

In [ ]:
def maskOceansKAM(lon, lat, var):
    # just a wrapper function for masking ocean
    
    lon2 = lon.copy()
    lon2[lon2 > 180] -= 360

    lons, lats = np.meshgrid(lon2, lat)
    var3 = maskoceans(lons, lats, var, resolution = 'l')
    lats = lats.reshape(np.shape(var3))
    var3 = np.ma.masked_where(lats < -60, var3) # and Antarctica
    
    return var3

### parameters of the model

In [ ]:
season = 'DJF' # 'DJF' or 'JJA' or 'MAM' or 'SON' or 'ALL'
# in CESM speak, rename obs to the same at command line using ncrename
# weird naming convention for Z3 because CESM field is all levels
varname = 'PSL' # 'TREFHT' or 'PSL' or 'Z3' or 'PREC' or 'T' or 'TS'
if varname == 'Z3':
    is3d = 1
    levUse = 500
elif varname == 'T':
    is3d = 1
    levUse = 1000
else:
    is3d = 0
    
# which dataset for the obs? 
# for temperature (TREFHT, T, or TS): 'BEST'
# for sea level pressure and geopotential height: '20CRV2c' or 'NCEP'
# for precip: 'GPCC'
obsname = '20CRV2c'  

# parameters for calculating interannual variability 
yrStart, yrEnd = 1921, 2014

# 20CRV2c ends in 2014
# NCEP starts in 1949
if obsname == '20CRV2c':
    if yrEnd > 2014:
        print 'Warning!! Years are being changed based on data availability. Proceed with caution.'
        yrEnd = 2014
if obsname == 'NCEP':
    if yrStart < 1949:
        print 'Warning!! Years are being changed based on data availability. Proceed with caution.'
        yrStart = 1949
yrs = np.arange(yrStart, yrEnd + 1, 1)
nyrs = np.shape(yrs)[0]
trendType1 = 'EM' # use the NCAR CESM1 ensemble mean as a basis for estimating the forced component

# parameters for assessing variability in trends
# i.e. may want to use full record for sampling internal variability, but then only examine
# a shorter period for the trend analysis
yrStartTrend, yrEndTrend = 1965, 2014
if obsname == '20CRV2c':
    if yrEndTrend > 2014:
        print 'Warning!! Years are being changed based on data availability. Proceed with caution.'
        yrEndTrend = 2014
if obsname == 'NCEP':
    if yrStartTrend < 1949:
        print 'Warning!! Years are being changed based on data availability. Proceed with caution.'
        yrStartTrend = 1949
yrsTrend = np.arange(yrStartTrend, yrEndTrend + 1, 1)
nyrsTrend = np.shape(yrsTrend)[0]
trendType2 = 'linear' # for summarizing spread across LENS and OLENS

# spatial domain
region = 'global'
if region == 'global':
    latRange = -90, 90
    lonRange = 0, 360

# bootstrapping params
nboot = 1000
blocksize = 2

# false discovery rate 
FDR = 0.1

# do a hash of relevant parameters
paramHash = hash(tuple((season, varname, obsname, yrStart, yrEnd, trendType1, yrStartTrend, yrEndTrend,\
                 trendType2, latRange, lonRange, nboot, blocksize)))

### file locations
Both output from the NCAR CESM1 LENS and various observational datasets are needed.  

Output from LENS is available on glade (yellowstone/cheyenne).  

All data are freely available online at URLs listed below.

In [ ]:
# LENS data
if varname == 'PREC': # have to combine large scale and convective
    modelDir = '/glade/p/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/monthly/' + varname + 'C'
    namesHist = sorted(glob.glob(modelDir + '/b.e11.B20TRC5CNBDRD.f09_g16*' + varname + '*200512.nc'))[:40] # last two runs not available for future
    namesFuture = sorted(glob.glob(modelDir + '/b.e11.BRCP85C5CNBDRD.f09_g16*' + varname + '*200601-*.nc'))
    
    modelDirL = '/glade/p/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/monthly/' + varname + 'L'
    namesHistL = sorted(glob.glob(modelDirL + '/b.e11.B20TRC5CNBDRD.f09_g16*' + varname + '*200512.nc'))[:40] # last two runs not available for future
    namesFutureL = sorted(glob.glob(modelDirL + '/b.e11.BRCP85C5CNBDRD.f09_g16*' + varname + '*200601-*.nc'))
else:
    modelDir = '/glade/p/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/monthly/' + varname
    namesHist = sorted(glob.glob(modelDir + '/b.e11.B20TRC5CNBDRD.f09_g16*' + varname + '*200512.nc'))[:40] # last two runs not available for future
    namesFuture = sorted(glob.glob(modelDir + '/b.e11.BRCP85C5CNBDRD.f09_g16*' + varname + '*200601-*.nc'))

# observational data
# BEST: http://berkeleyearth.lbl.gov/auto/Global/Gridded/Land_and_Ocean_LatLong1.nc
# 20CRV2c: https://www.esrl.noaa.gov/psd/data/gridded/data.20thC_ReanV2c.html
# slp: ftp://ftp.cdc.noaa.gov/Datasets/20thC_ReanV2c/Monthlies/monolevel/prmsl.mon.mean.nc
# geopotential height: ftp://ftp.cdc.noaa.gov/Datasets/20thC_ReanV2c/Monthlies/pressure/hgt.mon.mean.nc

dataDir = '' # INSERT DIRECTORY WHERE FILES WERE SAVED HERE
if obsname == 'BEST':
    dataName = 'Land_and_Ocean_LatLong1_' + varname + '.nc'
elif (obsname == '20CRV2c') & (varname == 'PSL'):
    dataName = 'prmsl.mon.mean.nc'
elif (obsname == '20CRV2c') & (varname == 'Z3'):
    dataName = 'hgt500.mon.mean.nc'
elif (obsname == 'NCEP') & (varname == 'PSL'):
    dataName = 'slp.mon.mean.nc'
elif (obsname == 'GPCC'):
    dataName = 'precip.mon.combined.total.v7.nc'

    
# rename obs to model speak
ds = Dataset(dataDir + dataName, 'r')
if (varname not in ds.variables.keys()):
#     import subprocess
#     subprocess.call('module load nco')
#     strUse = 'ncrename -v temperature,' + varname + ' ' + dataDir + dataName
#     subprocess.call(strUse, shell = True, executable = '/bin/bash')
# The above not working for unclear reasons
    print 'Change the varname in data to ' + varname
    raise KeyboardInterrupt

In [ ]:
# Change these paths to desired locations where output will be saved
outputDir = '' # WHERE SHOULD OUTPUT BE SAVED?
scratchDir = '' # WHERE SHOULD LARGE OUTPUT BE SAVED?
LENSoutput = '' # WHERE SHOULD INTERMEDIATE LENS FILES GO?
dataoutput = '' # WHERE SHOULD INTERMEDIATE DATA FILES GO?

### Time series of climate modes

Get observed and modeled time series of ENSO, PDO, and AMO

In [ ]:
def orthogonalize_timeseries(modeValNorm, nModes, nyrs):
# Create orthogonal time series from the mode time series

    from eofs.standard import Eof
    if nModes != np.shape(modeValNorm)[-1]:
        modeValNorm = np.transpose(modeValNorm)
        
    solver = Eof(modeValNorm)
    varExp = solver.varianceFraction(neigs=nModes)
    eofs = solver.eofs(neofs = nModes) 
    # Orthogonal time series
    # retain unity standard deviation
    modeValNormO = solver.pcs(npcs = nModes, pcscaling = 1)
    

    return modeValNormO

In [ ]:
def create_surrogates_1d(ts):
    '''
    Use the inverse fourier transform to shuffle phases.
    '''
    
    ts_fourier  = np.fft.fft(ts)
    
    if len(ts) % 2 == 1:
        random_phases = np.exp(np.random.uniform(0,np.pi,len(ts)/2+1)*1.0j)
        random_phases = np.hstack((-random_phases[::-1], random_phases[1:]))
    else:
        random_phases = np.exp(np.random.uniform(0,np.pi,len(ts)/2)*1.0j)
        random_phases = np.hstack((-random_phases[::-1], random_phases))
    ts_fourier_new = ts_fourier*random_phases
    new_ts = np.real(np.fft.ifft(ts_fourier_new))
    
    return new_ts

In [ ]:
# Observed time series
import requests

modeNames = 'MEI', 'PDO', 'AMO'
nModes = len(modeNames)
urlMEI = 'https://www.esrl.noaa.gov/psd/gcos_wgsp/Timeseries/Data/nino34.long.data'
urlPDO = 'http://research.jisao.washington.edu/pdo/PDO.latest.txt'
urlAMO = 'https://www.esrl.noaa.gov/psd/data/correlation/amon.us.long.data'

lagTime = 0 # in months

modeHash = hash(tuple((urlMEI, urlPDO, urlAMO, str(lagTime), str(yrs), season, modeNames)))
modeSaveName = outputDir + 'modeVals.' + str(modeHash) + '.npz'

# Get each of these indices for the correct season
if (not os.path.isfile(modeSaveName)):
    
    modeVal = np.empty((nyrs, len(modeNames)))
    for nn, nameUse in enumerate(modeNames):
        urlUse = eval('url' + nameUse)
        r = requests.get(urlUse)
        data = r.content
        data = data.split()
        if nameUse == 'MEI':
            headerLines = 2
            footerLines = -6
            ncolperrow = 13
        elif nameUse == 'AMO':
            headerLines = 2
            footerLines = -12
            ncolperrow = 13   
        elif nameUse == 'PDO':
            headerLines = 172
            footerLines = -50
            ncolperrow = 13

        data = np.array(data[headerLines:footerLines])
        nrows = np.shape(data)[0]/ncolperrow
        dataMat = np.reshape(data, (nrows, ncolperrow))

        # pull out relevant years and months to get seasonal averages 
        modeMonths = np.arange(1, 13)
        modeYears = dataMat[:, 0]
        if nameUse == 'PDO':
            modeYears = np.array([line.translate(None, '**') for line in modeYears])

        modeYears = modeYears.astype('int')
        nmodemonths, nmodeyears = np.shape(modeMonths)[0], np.shape(modeYears)[0]
        modeVals = dataMat[:, 1:].flatten().astype('float') # remove years

        # make matrices to pull out correct months and years
        modeMonths = np.repeat(modeMonths[np.newaxis,:], nmodeyears, axis = 0).flatten()
        modeYears = np.repeat(modeYears[:, np.newaxis], nmodemonths, axis = -1).flatten()

        # account for lag
        modeMonthsR = np.roll(modeMonths, -lagTime)
        modeYearsR = np.roll(modeYears, -lagTime) 


        for counter, yy in enumerate(yrs):

            if season == 'DJF':
                idxTime = ((modeMonthsR >= 1) & (modeMonthsR <= 2) & (modeYearsR == yy)) | \
                    ((modeMonthsR == 12) & (modeYearsR == yy-1))
            elif season == 'JJA':
                idxTime = ((modeMonthsR >= 6) & (modeMonthsR <= 8)) & (modeYearsR == yy)
            elif season == 'MAM':
                idxTime = ((modeMonthsR >= 3) & (modeMonthsR <= 5)) & (modeYearsR == yy)
            elif season == 'SON':
                idxTime = ((modeMonthsR >= 9) & (modeMonthsR <= 11)) & (modeYearsR == yy) 
            elif season == 'ALL':
                idxTime = (modeYearsR == yy)
            else:
                print 'Need to specify a canonical 3 month season, or ALL for full year'
                raise KeyboardInterrupt

            dummy = np.mean(modeVals[idxTime])
            normVal = np.std(dummy)
            modeVal[counter, nn] = dummy
            
    modeValNorm = (modeVal - np.mean(modeVal, axis = 0))/np.std(modeVal, axis = 0)
    
    modeValNormO = orthogonalize_timeseries(modeValNorm, nModes, nyrs)
    modeValNormO = (modeValNormO - np.mean(modeValNormO, axis = 0))/np.std(modeValNormO, axis = 0)
    # First EOF = like MEI and PDO
    # Second EOF = like AMO
    # Third EOF = remainders (most like MEI and PDO)
    
    np.savez(modeSaveName, modeValNorm = modeValNorm, modeValNormO = modeValNormO)

else:
    print 'Loading observational modes'
    ds = np.load(modeSaveName)
    modeValNormO = ds['modeValNormO'] # orthogonal time series
    modeValNorm = ds['modeValNorm']

In [ ]:
# Get modeled modes
modeDir = '/glade/p/cesmLE/CESM-CAM5-BGC-LE/CVDP/1920-2015/'

yrSpan = (str(modeDir.split('/')[-2]))[:4], (str(modeDir.split('/')[-2]))[5:]
# time in these files is in a non-python friendly format. Get from directory. 
time = pd.date_range(start = '1/' + yrSpan[0], end = '1/' + str(int(yrSpan[-1])+1), freq = 'M')

ENSOname = 'nino34'
PDOname = 'pdo_timeseries_mon'
AMOname = 'amo_timeseries_mon'

modeNames = 'ENSO', 'PDO', 'AMO'
nModes = len(modeNames)

if np.remainder(nyrs, 2) == 1:
    modeYrs = np.arange(yrStart - 1, yrEnd + 1, 1)
else:
    modeYrs = np.arange(yrStart, yrEnd + 1, 1)

ensMembers = np.hstack((np.arange(1, 36), np.arange(101, 106)))
nMembers = np.shape(ensMembers)[0]

modeHash = hash(tuple((modeDir, modeNames, ensMembers.tostring(), str(lagTime), str(yrs), season)))
modeSaveName = outputDir + 'LENS.modeVals.' + str(modeHash) + '.npz'

if (not os.path.isfile(modeSaveName)):

    modeValNorm_LENS = np.empty((nyrs, nModes, nMembers))
    modeValNormO_LENS = np.empty((nyrs, nModes, nMembers))

    for counter in np.arange(nMembers):
        fname = modeDir + 'CESM1-CAM5-BGC-LE_#' + str(ensMembers[counter]) + '.cvdp_data.1920-2015.nc'

        with Dataset(fname, 'r') as ds:

            modeVal = np.empty((nyrs, nModes))
            for mm, mode in enumerate(modeNames):
                varUse = eval(mode + 'name')
                modeVals = ds[varUse][:]

                # account for lag
                modeMonthsR = np.roll(time.month, -lagTime)
                modeYearsR = np.roll(time.year, -lagTime) 

                for jj, yy in enumerate(yrs):

                    if season == 'DJF':
                        idxTime = ((modeMonthsR >= 1) & (modeMonthsR <= 2) & (modeYearsR == yy)) | \
                            ((modeMonthsR == 12) & (modeYearsR == yy-1))
                    elif season == 'JJA':
                        idxTime = ((modeMonthsR >= 6) & (modeMonthsR <= 8)) & (modeYearsR == yy)
                    elif season == 'MAM':
                        idxTime = ((modeMonthsR >= 3) & (modeMonthsR <= 5)) & (modeYearsR == yy)
                    elif season == 'SON':
                        idxTime = ((modeMonthsR >= 9) & (modeMonthsR <= 11)) & (modeYearsR == yy) 
                    elif season == 'ALL':
                        idxTime = (modeYearsR == yy)
                    else:
                        print 'Need to specify a canonical 3 month season, or ALL for full year'
                        raise KeyboardInterrupt

                    dummy = np.mean(modeVals[idxTime])
                    normVal = np.std(dummy)
                    modeVal[jj, mm] = dummy

            modeValNorm = (modeVal - np.mean(modeVal, axis = 0))/np.std(modeVal, axis = 0)

            modeValNormO = orthogonalize_timeseries(modeValNorm, nModes, nyrs)
            modeValNormO = (modeValNormO - np.mean(modeValNormO, axis = 0))/np.std(modeValNormO, axis = 0)

            modeValNorm_LENS[:, :, counter] = modeValNorm
            modeValNormO_LENS[:, :, counter] = modeValNormO
    
    np.savez(modeSaveName, modeValNorm_LENS = modeValNorm_LENS, modeValNormO_LENS = modeValNormO_LENS)
    
else:
    
    ds = np.load(modeSaveName)
    modeValNorm_LENS = ds['modeValNorm_LENS']
    modeValNormO_LENS = ds['modeValNormO_LENS']
            


### calculate forced trend in model
Using approach of Dai et al (2015), assume that the locally forced temperature trend scales linearly with the global mean, ensemble mean temperature trend, and the analog for other variables.

In [ ]:
if (trendType1 == 'EM') | (trendType2 == 'EM'):
    
    emTrendFile = outputDir + 'cesm.EM.' + season + '.' + varname + '.' + str(yrStart) + '.' + str(yrEnd) + '.npy' 
    
    if os.path.isfile(emTrendFile):
        print 'Loading global mean, ensemble mean trend'
        EMglobal = np.load(emTrendFile)
        
    else:
    
        globalMeanTemp = np.empty((nyrs, len(namesHist)))

        for counter in np.arange(len(namesHist)):

            # read in historical simulation for first part
            ensNumber = (((namesHist[counter].split('/'))[-1]).split('.'))[4]

            print 'Calculating global average trend for ' + season + ' in CESM ens ' + ensNumber
            timeSpan = (((namesHist[counter].split('/'))[-1]).split('.'))[-2]
            fileBeginYr = (str(timeSpan)[:4])
            fileBeginMo = (str(timeSpan)[4:6])
            fileEndYr = int(str(timeSpan)[7:11])
            fileEndMo = int(str(timeSpan)[11:13])
            fid1 = Dataset(namesHist[counter], 'r')
            if varname == 'PREC':
                fid1L = Dataset(namesHistL[counter], 'r')
                
            time0 = fid1.variables['time'][:]
            ntime = time0.shape[0]
            time1 = pd.date_range(fileBeginMo + '/15/' + fileBeginYr, periods = ntime, freq = 'M')

            lat = fid1.variables['lat'][:]
            lon = fid1.variables['lon'][:]

            # load in values that are in the timespan
            idx1 = (time1.year >= yrStart - 1) & (time1.year <= yrEnd) # need one year earlier for the D in DJF

            if is3d:
                lev = fid1.variables['lev'][:]
                idxLev = np.argmin(np.abs(lev - levUse))
                T1 = (fid1.variables[varname][idx1, idxLev, :, :]).squeeze()
            elif varname == 'PREC':
                T1L = fid1L.variables[varname + 'L'][idx1, :, :] # large scale
                T1C = fid1.variables[varname + 'C'][idx1, :, :] # convective
                T1 = T1L + T1C
            else:
                T1 = fid1.variables[varname][idx1, :, :]

            # now read in future
            timeSpan = (((namesFuture[counter].split('/'))[-1]).split('.'))[-2]
            fileBeginYr = (str(timeSpan)[:4])
            fileBeginMo = (str(timeSpan)[4:6])
            fileEndYr = int(str(timeSpan)[7:11])
            fileEndMo = int(str(timeSpan)[11:13])

            fid2 = Dataset(namesFuture[counter], 'r')
            if varname == 'PREC':
                fid2L = Dataset(namesFutureL[counter], 'r') 
                
            time0 = fid2.variables['time'][:]
            ntime = time0.shape[0]
            time2 = pd.date_range(fileBeginMo + '/15/' + fileBeginYr, periods = ntime, freq = 'M')

            # load in values that are in the timespan
            idx2 = (time2.year >= yrStart - 1) & (time2.year <= yrEnd) # need one year earlier for the D in DJF

            if is3d:
                lev = fid2.variables['lev'][:]
                idxLev = np.argmin(np.abs(lev - levUse))
                T2 = (fid2.variables[varname][idx2, idxLev, :, :]).squeeze()
            elif varname == 'PREC':
                T2L = fid2L.variables[varname + 'L'][idx2, :, :] # large scale
                T2C = fid2.variables[varname + 'C'][idx2, :, :] # convective
                T2 = T2L + T2C
            else:
                T2 = fid2.variables[varname][idx2, :, :]

            T = np.concatenate((T1, T2), axis = 0)
            time = pd.date_range('1/1/' + str(yrStart - 1), periods = T.shape[0], freq = 'M')

            mlat = lat.shape[0]
            mlon = lon.shape[0]

            # take seasonal average 
            tempseasonal = np.empty((nyrs, mlat, mlon))
            timeseasonal = []
            for ct in np.arange(nyrs):
                yr = yrs[ct]
                if season == 'DJF':
                    idxTime = (((time.month >= 1) & (time.month <= 2)) & (time.year == yr)) | (((time.month == 12) & (time.year == yr - 1)))
                elif season == 'JJA':
                    idxTime = ((time.month >= 6) & (time.month <= 8)) & (time.year == yr)
                elif season == 'MAM':
                    idxTime = ((time.month >= 3) & (time.month <= 5)) & (time.year == yr)
                elif season == 'SON':
                    idxTime = ((time.month >= 9) & (time.month <= 11)) & (time.year == yr)
                elif season == 'ALL':
                    idxTime = (time.year == yr)
                else:
                    print 'Need to specify a canonical 3 month season'
                    raise KeyboardInterrupt
                tempseasonal[ct, :, :] = np.mean(T[idxTime, :, :], axis = 0)
                timeseasonal.append(pd.to_datetime((time[idxTime])[-1])) 

            # change units 
            if varname == 'PREC':
                modelUnits = fid1.variables[varname + 'C'].units
            else:
                modelUnits = fid1.variables[varname].units
                
            if modelUnits == 'K':
                tempseasonal -= 273.15
                modelUnits = 'degC'
            elif modelUnits == 'Pa':
                tempseasonal /= 100.
                modelUnits = 'hPa'
            elif ((modelUnits == 'm/s') & (varname == 'PREC')):
                # switch to monthly average precipitation for the season
                if season == 'DJF':
                    secPerMonth = 60*60*24*90/3.
                elif ((season == 'JJA') | (season == 'MAM')):
                    secPerMonth = 60*60*24*92/3.
                elif season == 'SON':
                    secPerMonth = 60*60*24*91/3.
                elif season == 'ALL':
                    secPerMonth = 60*60*24*365/12.
                    
                tempseasonal *= 1e3*secPerMonth # average cumulative precip per month, millimeters
                
            # calculate the global average value
            wgts = np.cos(lat*np.pi/180.)
            wgtsMat = (np.repeat(wgts[:, np.newaxis], mlon, axis = -1))[np.newaxis, :, :]
            globalMeanTemp[:, counter] = np.sum(tempseasonal.reshape((nyrs, mlat*mlon))*wgtsMat.reshape((1, mlat*mlon)), axis = -1)/np.sum(wgtsMat.flatten())

        # calculate average across simulations for forced signal
        EMglobal = np.mean(globalMeanTemp, axis = -1)

        # save 
        np.save(emTrendFile, EMglobal)


### put model and observations in same format (varnames, grids, etc.)

In [ ]:
# check which grid is coarser
fModel = Dataset(namesHist[0], 'r')
latModel = fModel.variables['lat'][:]
lonModel = fModel.variables['lon'][:]
avgDlatModel = np.mean(np.abs(latModel[1:] - latModel[:-1]))

fData = Dataset(dataDir + dataName, 'r')
try:
    latData = fData.variables['lat'][:]
    lonData = fData.variables['lon'][:]
except:
    latData = fData.variables['latitude'][:]
    lonData = fData.variables['longitude'][:]
    
avgDlatData = np.mean(np.abs(latData[1:] - latData[:-1]))

# Pull out units
obsUnits = fData.variables[varname].units


In [ ]:
import spharm

# create spharm objects for regridding
nlatM, nlonM = np.shape(latModel)[0], np.shape(lonModel)[0]
nlatD, nlonD = np.shape(latData)[0], np.shape(lonData)[0]

if ((obsname == 'BEST')) | ((obsname == '20CRV2c')) | (obsname == 'GPCC') | (obsname == 'NCEP'):
    spM = spharm.Spharmt(nlonM, nlatM, gridtype = 'regular', legfunc = 'computed')
    spD = spharm.Spharmt(nlonD, nlatD, gridtype = 'regular', legfunc = 'computed')
else:
    print 'Need to add functionality for other grids'
    raise KeyboardInterrupt

if avgDlatData > avgDlatModel:
    # regrid model to data grid
    print 'Regridding model to data grid'
    
    # first switch everything to 0,360
    lonData[lonData < 0] = lonData[lonData < 0] + 360
    idxLon = np.argsort(lonData) # remember to switch the data as well! 
    lon = np.sort(lonData)
        
    # change data latitude to increasing if it is not
    idxLat = np.argsort(latData) # remember to switch the data as well! 
    lat = np.sort(latData)
        
    # check that latitude is increasing, and longitude is greater than zero
    if (np.min(lon) < 0) | (np.min(lonModel) < 0) | (lat[-1] < lat[0]) | (latModel[-1] < latModel[0]):
        raise KeyboardInterrupt
    
    nlat, nlon = np.shape(lat)[0], np.shape(lon)[0]
    
    # take seasonal averages for the data
    saveNameObs = dataoutput + obsname + '.' + season + '.' + str(yrStart) + '.' + str(yrEnd) + '.' + varname + '.nc'
    
    if (not os.path.isfile(saveNameObs)):
    
        time = fData.variables['time'][:]
        if obsname == 'BEST':
            year = time.astype(int)
            month = np.ceil((time - year)*12).astype(int)
        elif ((obsname == '20CRV2c') | (obsname == 'GPCC')) | (obsname == 'NCEP'):
            from netcdftime import utime
            cdftime = utime(fData.variables['time'].units)
            t = cdftime.num2date(fData.variables['time'][:])
            year = np.array([(t[counter]).year for counter in np.arange(np.shape(t)[0])])
            month = np.array([(t[counter]).month for counter in np.arange(np.shape(t)[0])])

        idx1 = (year >= yrStart - 1) & (year <= yrEnd) # need one year earlier for the D in DJF
        T = fData.variables[varname][idx1, :, :]
        T = T[:, :, idxLon] # re-order the longitudes 
        T = T[:, idxLat, :] # and latitudes if necessary

        # put into pandas
        time = pd.date_range(start = '1/1/' + str(yrStart - 1), periods = np.shape(T)[0], freq = 'M')
        tempseasonal = np.empty((nyrs, nlat, nlon))
        timeseasonal = []
        for ct in np.arange(nyrs):
            yr = yrs[ct]
            if season == 'DJF':
                idxTime = (((time.month >= 1) & (time.month <= 2)) & (time.year == yr)) | \
                    (((time.month == 12) & (time.year == yr - 1)))
            elif season == 'JJA':
                idxTime = ((time.month >= 6) & (time.month <= 8)) & (time.year == yr)
            elif season == 'MAM':
                idxTime = ((time.month >= 3) & (time.month <= 5)) & (time.year == yr)
            elif season == 'SON':
                idxTime = ((time.month >= 9) & (time.month <= 11)) & (time.year == yr)
            elif season == 'ALL':
                idxTime = (time.year == yr)
            else:
                print 'Need to specify a canonical 3 month season'
                raise KeyboardInterrupt

            tempseasonal[ct, :, :] = np.mean(T[idxTime, :, :], axis = 0)
            timeseasonal.append(pd.to_datetime((time[idxTime])[-1]))

        # change to hPa
        if obsUnits == 'Pa':
            tempseasonal /= 100.
            obsUnits = 'hPa'
        elif obsUnits == 'millibars':
            obsUnits = 'hPa'
            
        # save to netcdf
        fout = Dataset(saveNameObs, 'w')

        fout.createDimension('lat', nlat)
        fout.createDimension('lon', nlon)
        fout.createDimension('time', 0)

        latnc = fout.createVariable('lat', 'f8', ('lat',))
        lonnc = fout.createVariable('lon', 'f8', ('lon',))
        timenc = fout.createVariable('time', 'f8', ('time',))
        tempnc = fout.createVariable(varname, 'f8', ('time','lat','lon'))

        fout.description = season + ' average ' + varname + ' from ' + obsname

        latnc.units = 'degree_north'
        lonnc.units = 'degree_east'
        tempnc.units = obsUnits
        timenc.units = 'days since 1850-01-01 00:00:00'
        timenc.calendar = 'gregorian'

        latnc[:] = lat
        lonnc[:] = lon
        tempnc[:, :, :] = tempseasonal

        from datetime import datetime, timedelta
        from netCDF4 import num2date, date2num

        timenc[:] = date2num(timeseasonal, units = timenc.units)

        fout.close()   
    
    
    for counter in np.arange(len(namesHist)):

        # read in historical simulation for first part
        ensNumber = (((namesHist[counter].split('/'))[-1]).split('.'))[4]
        
        # name to save regridded output
        saveName = LENSoutput + 'LENS.' + season + '.' + str(yrStart) + '.' + str(yrEnd) + '.' + varname + '.regridded.to.' + obsname + '.' + ensNumber + '.nc'
        
        if (not os.path.isfile(saveName)):
            print 'Taking seasonal averages and regridding member ' + ensNumber

            timeSpan = (((namesHist[counter].split('/'))[-1]).split('.'))[-2]
            fileBeginYr = (str(timeSpan)[:4])
            fileBeginMo = (str(timeSpan)[4:6])
            fileEndYr = int(str(timeSpan)[7:11])
            fileEndMo = int(str(timeSpan)[11:13])
            fid1 = Dataset(namesHist[counter], 'r')
            time0 = fid1.variables['time'][:]
            ntime = time0.shape[0]
            time1 = pd.date_range(fileBeginMo + '/15/' + fileBeginYr, periods = ntime, freq = 'M')

            if varname == 'PREC':
                modelUnits = fid1.variables[varname + 'C'].units
            else:
                modelUnits = fid1.variables[varname].units
            # load in values that are in the timespan
            idx1 = (time1.year >= yrStart - 1) & (time1.year <= yrEnd) # need one year earlier for the D in DJF

            if is3d:
                lev = fid1.variables['lev'][:]
                idxLev = np.argmin(np.abs(lev - levUse))
                T1 = (fid1.variables[varname][idx1, idxLev, :, :]).squeeze()
            elif varname == 'PREC':
                fid1L = Dataset(namesHistL[counter], 'r')
                T1L = fid1L.variables[varname + 'L'][idx1, :, :] # large scale
                T1C = fid1.variables[varname + 'C'][idx1, :, :] # convective
                T1 = T1L + T1C
            else:
                T1 = fid1.variables[varname][idx1, :, :]

            # now read in future
            timeSpan = (((namesFuture[counter].split('/'))[-1]).split('.'))[-2]
            fileBeginYr = (str(timeSpan)[:4])
            fileBeginMo = (str(timeSpan)[4:6])
            fileEndYr = int(str(timeSpan)[7:11])
            fileEndMo = int(str(timeSpan)[11:13])

            fid2 = Dataset(namesFuture[counter], 'r')
            time0 = fid2.variables['time'][:]
            ntime = time0.shape[0]
            time2 = pd.date_range(fileBeginMo + '/15/' + fileBeginYr, periods = ntime, freq = 'M')

            # load in values that are in the timespan
            idx2 = (time2.year >= yrStart - 1) & (time2.year <= yrEnd) # need one year earlier for the D in DJF

            if is3d:
                lev = fid2.variables['lev'][:]
                idxLev = np.argmin(np.abs(lev - levUse))
                T2 = (fid2.variables[varname][idx2, idxLev, :, :]).squeeze()
            elif varname == 'PREC':
                fid2L = Dataset(namesFutureL[counter], 'r')
                T2L = fid2L.variables[varname + 'L'][idx2, :, :] # large scale
                T2C = fid2.variables[varname + 'C'][idx2, :, :] # convective
                T2 = T2L + T2C
            else:
                T2 = fid2.variables[varname][idx2, :, :]

            T = np.concatenate((T1, T2), axis = 0)

            time = pd.date_range('1/1/' + str(yrStart - 1), periods = T.shape[0], freq = 'M')
            
            tempseasonal = np.empty((nyrs, nlat, nlon))
            timeseasonal = []
            for ct in np.arange(nyrs):
                yr = yrs[ct]
                if season == 'DJF':
                    idxTime = (((time.month >= 1) & (time.month <= 2)) & (time.year == yr)) | \
                        (((time.month == 12) & (time.year == yr - 1)))
                elif season == 'JJA':
                    idxTime = ((time.month >= 6) & (time.month <= 8)) & (time.year == yr)
                elif season == 'MAM':
                    idxTime = ((time.month >= 3) & (time.month <= 5)) & (time.year == yr)
                elif season == 'SON':
                    idxTime = ((time.month >= 9) & (time.month <= 11)) & (time.year == yr)
                elif season == 'ALL':
                    idxTime = (time.year == yr)
                else:
                    print 'Need to specify a canonical 3 month season'
                    raise KeyboardInterrupt
                    
                dummy = np.mean(T[idxTime, :, :], axis = 0)

                # regrid(grid in, grid out)
                data_interp = spharm.regrid(spM, spD, dummy)

                tempseasonal[ct, :, :] = data_interp
                timeseasonal.append(pd.to_datetime((time[idxTime])[-1])) 
            
            # change units for temperature
            if modelUnits == 'K':
                tempseasonal -= 273.15
                modelUnits = 'degC'
            elif modelUnits == 'Pa':
                tempseasonal /= 100.
                modelUnits = 'hPa'
            elif ((modelUnits == 'm/s') & (varname == 'PREC')):
                # switch to monthly average precipitation for the season
                if season == 'DJF':
                    secPerMonth = 60*60*24*90/3.
                elif ((season == 'JJA') | (season == 'MAM')):
                    secPerMonth = 60*60*24*92/3.
                elif season == 'SON':
                    secPerMonth = 60*60*24*91/3.
                elif season == 'ALL':
                    secPerMonth = 60*60*24*365/12.
                    
                tempseasonal *= 1e3*secPerMonth # average cumulative precip per month, millimeters
                modelUnits = 'mm/mn' # mm/month to match obs
            elif modelUnits == 'm':
                print 'Units all good!'
            else:
                print 'check whether units need to be changed'
                raise KeyboardInterrupt
                
            # save to netcdf
            fout = Dataset(saveName, 'w')

            fout.createDimension('lat', nlat)
            fout.createDimension('lon', nlon)
            fout.createDimension('time', 0)

            latnc = fout.createVariable('lat', 'f8', ('lat',))
            lonnc = fout.createVariable('lon', 'f8', ('lon',))
            timenc = fout.createVariable('time', 'f8', ('time',))
            tempnc = fout.createVariable(varname, 'f8', ('time','lat','lon'))

            fout.description = season + ' average ' + varname + ' from LENS regridded to ' + obsname + ' grid'

            latnc.units = 'degree_north'
            lonnc.units = 'degree_east'
            tempnc.units = modelUnits
            timenc.units = 'days since 1850-01-01 00:00:00'
            timenc.calendar = 'gregorian'

            latnc[:] = latData
            lonnc[:] = lonData
            tempnc[:, :, :] = tempseasonal

            from datetime import datetime, timedelta
            from netCDF4 import num2date, date2num

            timenc[:] = date2num(timeseasonal, units = timenc.units)

            fout.close()         
    
else:
    # regrid data to model grid
    print 'Not currently an option to regrid data to model grid'
    raise KeyboardInterrupt

In [ ]:
# Create ensemble mean
fNamesMembers = sorted(glob.glob(LENSoutput + 'LENS.' + season + '.' + str(yrStart) + '.' + str(yrEnd) + '.' + varname + '.regridded.to.' + obsname + '*[^0-9]*.nc'))
fNameEM = LENSoutput + 'LENS.' + season + '.' + str(yrStart) + '.' + str(yrEnd) + '.' + varname + '.regridded.to.' + obsname + '.EM.nc'

nlat, nlon = lat.shape[0], lon.shape[0]

if (not os.path.isfile(fNameEM)):

    EM = np.empty((nyrs, nlat, nlon, len(fNamesMembers)))
    for counter, f in enumerate(fNamesMembers):
        ds = Dataset(f, 'r')
        EM[:, :, :, counter] = ds.variables[varname][:]

    modelUnits = ds.variables[varname].units

    time = ds.variables['time'][:]
    EM = np.mean(EM, axis = -1)
    fout = Dataset(fNameEM, 'w')

    fout.createDimension('lat', nlat)
    fout.createDimension('lon', nlon)
    fout.createDimension('time', 0)

    latnc = fout.createVariable('lat', 'f8', ('lat',))
    lonnc = fout.createVariable('lon', 'f8', ('lon',))
    timenc = fout.createVariable('time', 'f8', ('time',))
    tempnc = fout.createVariable(varname, 'f8', ('time','lat','lon'))

    fout.description = season + ' average temperatures from CESM LE EM'

    latnc.units = 'degree_north'
    lonnc.units = 'degree_east'
    tempnc.units = modelUnits
    timenc.units = 'days since 1850-01-01 00:00:00'
    timenc.calendar = 'gregorian'

    latnc[:] = lat
    lonnc[:] = lon
    tempnc[:, :, :] = EM
    timenc[:] = time

    fout.close()

### Bootstrapping

Model: T$_t$ = $\beta_0$ + $\beta_1$ x trend$_t$ + $\beta_{2,n}$ x modes$_t$ + $\epsilon_t$

Estimate all parameters from the full time series (e.g. 1921-2015, or set by user)

Create synthetic time series by randomly generating new versions of $\beta_{2,n}$ x modes$_t$ + $\epsilon_t$ while keeping the first two terms of the model, $\beta_0$ + $\beta_1$ x trend$_t$, fixed across all samples

The modes term is generated by creating surrogate versions of modes$_t$ via phase shuffling.

The $\epsilon_t$ term is bootstrapped


In [ ]:
# file to save bootstrapping output
# putting in scratch space because can be big, and doesn't take that long to re-run

useModes = 1
bootOutputName = scratchDir + 'bootstrap.modes.' + str(useModes) + '.' + str(paramHash) + '.npz'
# save statistics of bootstrap samples here
bootOutputSummaryName = outputDir + 'bootstrapSummary.modes.' + str(useModes) + '.' + str(paramHash) + '.npz'
# save the modified bootstrap output -- modes or bootstrap only
bootOutputNameMod1 = scratchDir + 'bootstrap.modes.' + str(useModes) + '.' + str(paramHash) + '.mod1.npz'
bootOutputNameMod2 = scratchDir + 'bootstrap.modes.' + str(useModes) + '.' + str(paramHash) + '.mod2.npz'

#obsBootOutputName = scratchDir + 'bootstrap.modes.obs.' + str(paramHash) + '.npz'
#obsBootOutputSummaryName = outputDir + 'bootstrapSummary.modes.obs.' + str(paramHash) + '.npz'
     
# stack files such that the order is EM, all model members, then obs
fnames = list(fNamesMembers)
fnames.insert(0, fNameEM)
fnames.append(saveNameObs)


# Stack mode matrices
modeValsAll = np.dstack((np.mean(modeValNormO_LENS, axis = -1)[:, :, np.newaxis]
                        , modeValNormO_LENS, modeValNormO[:, :, np.newaxis]))

print bootOutputName
print bootOutputSummaryName


In [ ]:
yrs0 = yrs - np.mean(yrs)
yrsTrend0 = yrsTrend - np.mean(yrsTrend)

# what trend should be removed when calculating interannual variability?
if trendType1 == 'linear': 
    X = np.matrix(np.hstack((np.ones((nyrs, 1)), yrs0[:, np.newaxis])))
elif trendType1 == 'quad':
    X = np.matrix(np.hstack((np.ones((nyrs, 1)), yrs0[:, np.newaxis], (yrs0[:, np.newaxis])**2)))
elif trendType1 == 'EM': 
    EM = np.load(emTrendFile)
    EM_anom = EM - np.mean(EM)
    X = np.matrix(np.hstack((np.ones((nyrs, 1)), EM_anom[:, np.newaxis])))
    
# what should be used as a covariate in calculating trends?
if trendType2 == 'linear': 
    X0 = np.matrix(np.hstack((np.ones((nyrsTrend, 1)), yrsTrend0[:, np.newaxis])))
elif trendType2 == 'quad':
    X0 = np.matrix(np.hstack((np.ones((nyrsTrend, 1)), yrsTrend0[:, np.newaxis], (yrsTrend0[:, np.newaxis])**2)))
elif trendType2 == 'EM': 
    EM = np.load(emTrendFile)
    # pull out matching period
    EMsub = EM[np.in1d(yrs, yrsTrend)]
    EM_anom = EMsub - np.mean(EMsub)
    X0 = np.matrix(np.hstack((np.ones((nyrsTrend, 1)), EM_anom[:, np.newaxis])))

# set to unity standard deviation 
X[:, 1:] = (X[:, 1:] - np.mean(X[:, 1:], axis = 0))/np.std(X[:, 1:], axis = 0)

In [ ]:
# Need to change temp directory in order to save large numpy file to a place that has lots of space 
import tempfile
tempfile.tempdir = scratchDir


In [ ]:
# Examine temporal properties of residuals for OLENS
f = fnames[-1]
ds = Dataset(f, 'r') 
T = ds.variables[varname][:]
if obsname == 'Hadley': # missing data case
    T = np.ma.masked_where(np.isnan(T), T)
Tvec = np.matrix(T.reshape((nyrs, nlon*nlat))) # time by space

if useModes:
    modeTS = modeValsAll[:, :, -1]
    Xnew = np.hstack((X, modeTS))

else:
    Xnew = X.copy()

# trend as a function of predictor (time or EM temperature trend) in order to estimate residuals
# Modeling both the forced trend and the modes here
beta = (np.dot(np.dot((np.dot(Xnew.T, Xnew)).I, Xnew.T), Tvec)) 

trendEst = np.dot(Xnew, beta) # best fit to predictor
resVals = np.array(Tvec - trendEst) # residuals (to be bootstrapped)
oceanmask = np.ma.getmask(maskOceansKAM(lon, lat, resVals[0,:]))


In [ ]:
lons, lats = np.meshgrid(lon, lat)
lons, lats = lons.flatten(), lats.flatten()
from pypr.stattest.ljungbox import *
idxLand = np.where((~oceanmask )& (lats > -60))[0]
LBstat = 2*np.ones((nlat*nlon,))
for ii in idxLand:
    x = resVals[:, ii]
    h, pV, Q, cV = lbqtest(x, range(20,21), alpha = 0.05)
    LBstat[ii] = h[0]
    
wgts = np.sqrt(np.cos(np.deg2rad(lats)))
idxWhite = 1 - np.sum(wgts[idxLand]*LBstat[idxLand])/np.sum(wgts)
print str(np.round(idxWhite, 2)) + ' fraction of land area is white noise for ' + varname + ' in ' + season

In [ ]:
# Save members of OLENS 

# set seed so can compare, e.g., PSL and TREFHT
np.random.seed(123)

# Load in observations
ds = Dataset(saveNameObs, 'r') 
T = ds.variables[varname][:]
if obsname == 'Hadley': # missing data case
    T = np.ma.masked_where(np.isnan(T), T)
Tvec = np.matrix(T.reshape((nyrs, nlon*nlat))) # time by space

# Get predictors (constant, forced component, trend)
modeTS = modeValsAll[:, :, -1]
Xnew = np.hstack((X, modeTS))

# trend as a function of predictor (time or EM temperature trend) in order to estimate residuals
# Modeling both the forced trend and the modes here
beta = (np.dot(np.dot((np.dot(Xnew.T, Xnew)).I, Xnew.T), Tvec)) 

trendEst = np.dot(Xnew, beta) # best fit to predictor
resVals = Tvec - trendEst # residuals (to be bootstrapped)

SAMPLES = np.empty((nyrs, nlat*nlon, nboot))

trendEst0 = np.dot(X, beta[:2, :])

for kk in np.arange(nboot):

    if np.mod(nyrs, blocksize) == 0:

        numSeeds = nyrs/blocksize
        sampleIdx = np.random.randint(0, nyrs, numSeeds)
        idxUse = (np.array([np.arange(sampleIdx[jj], sampleIdx[jj] + blocksize) for jj in np.arange(np.shape(sampleIdx)[0])])).ravel()
        idxUse = np.mod(idxUse, nyrs)

    else:

        numSeeds = nyrs/blocksize
        sampleIdx = np.random.randint(0, nyrs, numSeeds)
        idxUse = (np.array([np.arange(sampleIdx[jj], sampleIdx[jj] + blocksize) for jj in np.arange(np.shape(sampleIdx)[0])])).ravel()
        timeLeft = np.mod(nyrs, blocksize)
        sampleLeftover = np.random.randint(0, nyrs, 1) # just going to have one set of randoms 
        idxUse2 = np.arange(sampleLeftover, sampleLeftover + timeLeft)
        idxUse = np.append(idxUse, idxUse2)
        idxUse = np.mod(idxUse, nyrs)


    # create surrogate versions of the modes
    surrogateMode = np.empty_like(modeTS)
    for mm in np.arange(modeTS.shape[-1]):
        ts = modeValNormO[:, mm]
        dummy = create_surrogates_1d(ts)
        # do adjustment so overall variance matches
        dummy *= np.std(ts)/np.std(dummy)
        surrogateMode[:, mm] = dummy

    trendEstModes = np.dot(surrogateMode, beta[-nModes:, :])
    SAMPLES[:, :, kk] = trendEst0 + trendEstModes + resVals[idxUse, :]

In [ ]:
# Recenter samples on LENS EM for OLENS
dsEM = Dataset(fNameEM, 'r')
EMvals = dsEM[varname][:]
EMvals = EMvals.reshape((nyrs, nlat*nlon))
OLENS = (SAMPLES - np.mean(SAMPLES, -1)[:, :, np.newaxis] + EMvals[:, :, np.newaxis]).reshape((nyrs, nlat, nlon, nboot))

# Define units
if ((varname == 'TREFHT') | (varname == 'T') | (varname == 'TS')):
    unitsName = 'degC'
elif varname == 'PSL':
    unitsName = 'hPa'
elif varname == 'Z3':
    unitsName = 'm'
elif varname == 'PREC':
    unitsName = 'mm/mo'
    

In [ ]:
# Save to a netcdf
# Need to divide into 10 files in order to fit into dataverse
nfiles = 20
samples_per_file = nboot/nfiles

for filect in np.arange(nfiles):

    savename = '/glade/scratch/mckinnon/OLENS/%s_%s_%i-%i_file%02d.nc' \
        % (varname, season, yrs[0], yrs[-1], filect + 1)
        
    print(savename)
    time = dsEM.variables['time'][:]
    fout = Dataset(savename, 'w')
    
    samplesSave = np.arange(filect*samples_per_file, samples_per_file*(filect+1))

    fout.createDimension('lat', nlat)
    fout.createDimension('lon', nlon)
    fout.createDimension('samples', samples_per_file)
    fout.createDimension('time', 0)

    latnc = fout.createVariable('lat', 'f8', ('lat',))
    lonnc = fout.createVariable('lon', 'f8', ('lon',))
    timenc = fout.createVariable('time', 'f8', ('time',))
    samplenc = fout.createVariable('samples', 'i', ('samples',))
    tempnc = fout.createVariable(varname, 'f8', ('time','lat','lon', 'samples'))

    fout.description = ("Samples of %s %s from the Observational Large Ensemble (OLENS). "
                        "Data is from %s. "  
                       "The forced component of OLENS is the ensemble mean of 40 members of the "
                       "NCAR CESM1 Large Ensemble. "
                       "See McKinnon and Deser, J Clim, for more details." % (season, varname, obsname))

    latnc.units = 'degree_north'
    lonnc.units = 'degree_east'
    tempnc.units = unitsName
    timenc.units = 'days since 1850-01-01 00:00:00'
    timenc.calendar = 'gregorian'

    latnc[:] = lat
    lonnc[:] = lon
    samplenc[:] = samplesSave + 1
    tempnc[:, :, :, :] = OLENS[:, :, :, filect*samples_per_file:samples_per_file*(filect+1)]
    timenc[:] = time

    fout.close()


In [ ]:
# Perform bootstrapping, and then calculate trends across samples over desired period

if (not os.path.isfile(bootOutputName)): # if not already done

    # set seed so can compare, e.g., PSL and TREFHT
    np.random.seed(123)

    BETA = np.empty((len(fnames), nlat*nlon)) # empirical trends (actual)
    BOOTSAMPLES = np.empty((len(fnames), nlat*nlon, nboot)) # bootstrapped trends
    BOOTSAMPLES_MOD1 = BOOTSAMPLES.copy()
    BOOTSAMPLES_MOD2 = BOOTSAMPLES.copy()
    INTERANNUALVAR = np.empty((len(fnames), nlat*nlon)) # variance across time
    empiricalAR1 = np.empty((len(fnames), nlat*nlon)) # estimated AR(1) coefficient
    
    for ii, f in enumerate(fnames):

        print f

        ds = Dataset(f, 'r')
        T = ds.variables[varname][:]
        if obsname == 'Hadley': # missing data case
            T = np.ma.masked_where(np.isnan(T), T)
        Tvec = np.matrix(T.reshape((nyrs, nlon*nlat))) # time by space
        
        if useModes:
            modeTS = modeValsAll[:, :, ii]
            Xnew = np.hstack((X, modeTS))

        else:
            Xnew = X.copy()
        
        # trend as a function of predictor (time or EM temperature trend) in order to estimate residuals
        # Modeling both the forced trend and the modes here
        beta = (np.dot(np.dot((np.dot(Xnew.T, Xnew)).I, Xnew.T), Tvec)) 
        
        Tvecsub = Tvec[np.in1d(yrs, yrsTrend), :]
        beta0 = (np.dot(np.dot((np.dot(X0.T, X0)).I, X0.T), Tvecsub)) # trend over period of interest
        BETA[ii, :] = beta0[-1, :]

        trendEst = np.dot(Xnew, beta) # best fit to predictor
        resVals = Tvec - trendEst # residuals (to be bootstrapped)

        
        # trend estimate from intercept and trend model only (no modes)
        trendEst0 = np.dot(X, beta[:2, :])
        
        # calculate autocorrelation for lag 1
        empiricalAR1[ii, :] = np.array([autocorr(np.array(resVals[:, gg]).flatten()) for gg in np.arange(nlat*nlon)])[:, 1]
        
        # calculate year-to-year variance
        INTERANNUALVAR[ii, :] = np.var(resVals, axis = 0, ddof = 2) # 2 less degrees of freedom: mean and trend
        
        for kk in np.arange(nboot):
            
            if np.mod(nyrs, blocksize) == 0:
                
                numSeeds = nyrs/blocksize
                sampleIdx = np.random.randint(0, nyrs, numSeeds)
                idxUse = (np.array([np.arange(sampleIdx[jj], sampleIdx[jj] + blocksize) for jj in np.arange(np.shape(sampleIdx)[0])])).ravel()
                idxUse = np.mod(idxUse, nyrs)

            else:
                
                numSeeds = nyrs/blocksize
                sampleIdx = np.random.randint(0, nyrs, numSeeds)
                idxUse = (np.array([np.arange(sampleIdx[jj], sampleIdx[jj] + blocksize) for jj in np.arange(np.shape(sampleIdx)[0])])).ravel()
                timeLeft = np.mod(nyrs, blocksize)
                sampleLeftover = np.random.randint(0, nyrs, 1) # just going to have one set of randoms 
                idxUse2 = np.arange(sampleLeftover, sampleLeftover + timeLeft)
                idxUse = np.append(idxUse, idxUse2)
                idxUse = np.mod(idxUse, nyrs)
                
                
            # create surrogate versions of the modes
            if useModes:
                surrogateMode = np.empty_like(modeTS)
                for mm in np.arange(modeTS.shape[-1]):
                    ts = modeValNormO[:, mm]
                    dummy = create_surrogates_1d(ts)
                    # do adjustment so overall variance matches
                    dummy *= np.std(ts)/np.std(dummy)
                    surrogateMode[:, mm] = dummy
                
                trendEstModes = np.dot(surrogateMode, beta[-nModes:, :])
                bootsample = trendEst0 + trendEstModes + resVals[idxUse, :]

                # Do experiment: how much of variability is due to modes, versus due to atmospheric variability
                # that is captured in the bootstrapping? 
                # Do modified bootsamples that only include one or the other
                bootsampleMod1 = trendEst0 + trendEstModes # only modes
                bootsampleMod2 = trendEst0 + resVals[idxUse, :] # exclude modes
            else:
                bootsample = trendEst0 + resVals[idxUse, :]
                

            bootsampleSub = bootsample[np.in1d(yrs, yrsTrend), :]
            BOOTSAMPLES[ii, :, kk] = np.array((np.dot(np.dot((np.dot(X0.T, X0)).I, X0.T), bootsampleSub)))[-1, :]
            
            if useModes:
                bootsampleSubMod1 = bootsampleMod1[np.in1d(yrs, yrsTrend), :]
                bootsampleSubMod2 = bootsampleMod2[np.in1d(yrs, yrsTrend), :]
                BOOTSAMPLES_MOD1[ii, :, kk] = np.array((np.dot(np.dot((np.dot(X0.T, X0)).I, X0.T), bootsampleSubMod1)))[-1, :]
                BOOTSAMPLES_MOD2[ii, :, kk] = np.array((np.dot(np.dot((np.dot(X0.T, X0)).I, X0.T), bootsampleSubMod2)))[-1, :]
            

    BOOTSTD = np.std(BOOTSAMPLES, axis = -1)
    np.savez_compressed(bootOutputSummaryName, BETA = BETA, BOOTSTD = BOOTSTD, \
                        INTERANNUALVAR = INTERANNUALVAR,empiricalAR1 = empiricalAR1)
    
    np.savez_compressed(bootOutputName, BETA = BETA, BOOTSAMPLES = BOOTSAMPLES)
    
    if useModes:
        np.savez_compressed(bootOutputNameMod1, BOOTSAMPLES_MOD1 = BOOTSAMPLES_MOD1)
        np.savez_compressed(bootOutputNameMod2, BOOTSAMPLES_MOD2 = BOOTSAMPLES_MOD2)
    
else:
    print "Loading bootstrap output"
    # print "Not loading bootstrap samples for now, but load later for plots"
    loadName = bootOutputSummaryName
    f = np.load(loadName)
    BETA = f['BETA']
    if loadName == bootOutputName:
        BOOTSAMPLES = f['BOOTSAMPLES']
    else:
        BOOTSTD = f['BOOTSTD']
        INTERANNUALVAR = f['INTERANNUALVAR']
        empiricalAR1 = f['empiricalAR1']